In [ ]:
import tensorflow as tf
import numpy as np
AUTO = tf.data.experimental.AUTOTUNE
GCS_OUTPUT = 'gs://cloudfire_citric-sol_5670/fire_dataset/tfrecords-dataset-5x/'  # prefix for output file names

TARGET_SIZE = [224, 224]


def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means scalar

        # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "label": tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "size": tf.io.FixedLenFeature([2], tf.int64),  # two integers
        "one_hot_class": tf.io.VarLenFeature(tf.float32)  # a certain number of floats
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)

    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding

    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.reshape(image, [*TARGET_SIZE, 3])

    class_num = example['class']

    label = example['label']
    height = example['size'][0]
    width = example['size'][1]
    one_hot_class = tf.sparse.to_dense(example['one_hot_class'])
    return image, class_num, label, height, width, one_hot_class


# read from TFRecords. For optimal performance, read from multiple
# TFRecord files at once and set the option experimental_deterministic = False
# to allow order-altering optimizations.

option_no_order = tf.data.Options()
option_no_order.experimental_deterministic = False

filenames = tf.io.gfile.glob(GCS_OUTPUT + "*.tfrec")
dataset4 = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
dataset4 = dataset4.with_options(option_no_order)
dataset4 = dataset4.map(read_tfrecord, num_parallel_calls=AUTO)

In [ ]:
dataset4

In [ ]:
for image, class_num, label, height, width, one_hot_class in dataset4.take(100):
    print("Image shape {}, {}x{} px, class={} ({:>10}, {})".format(image.numpy().shape, width, height, class_num,label.numpy().decode('utf8'),one_hot_class))

In [ ]:
from matplotlib import pyplot as plt
CLASSES = ['Fire', 'Normal']

def dataset_to_numpy_util(dataset, N):
    dataset = dataset.batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels
'''
def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct
'''
def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1
  
def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = CLASSES[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()  

    '''
def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()
    
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])
    '''


In [ ]:
display_9_images_from_dataset(dataset4)

In [ ]:
from matplotlib import pyplot as plt
CLASSES = ['Fire', 'Normal']

def show_batch(numpy_images, numpy_labels):
    plt.figure(figsize=(10,10))
    for n in range(25):
        plt.subplot(5, 5, n+1)
        plt.title(CLASSES[np.argmax(numpy_labels[n], axis=0)])
        plt.imshow(numpy_images[n])
        plt.axis('off')

In [ ]:
def dataset_to_numpy_util(dataset4,N):
    dataset = dataset4.batch(N)
    for image, _, label, _, _, _ in dataset:
        numpy_images = image.numpy()
        numpy_labels = label.numpy() 
        show_batch(numpy_images, numpy_labels)      

In [ ]:
dataset_to_numpy_util(dataset4, 25) 

In [ ]:
%%bash
pip install kaggle

In [ ]:
%%bash
gsutil -m cp -r /home/jupyter/FYP_FIRENET/fire-eye gs://cloudfire_lyrical-edition-273206/trained

### Local Train

In [ ]:
%%bash
MODEL_DIR=/home/jupyter/FYP_FIRENET/fire-eye-test77
rm -rf Trainer.tar.gz fire-eye
gcloud ai-platform local train \
    --module-name=Trainer.config \
    --package-path=${PWD}/Trainer \
    -- \
    --output_dir=$MODEL_DIR\
    --train_steps=500 \
    --learning_rate=0.001 \
    --batch_size=20 \
    --train_data_path=gs://cloudfire_citric-sol_5670/fire_dataset/tfrecords-dataset-*x/ \
    --eval_data_path=gs://cloudfire_citric-sol_5670/fire_dataset/tfrecords-dataset-eval/

In [3]:
%%bash
tensorboard --logdir /home/jupyter/FYP_FIRENET/fire-eye-test77 --port 8000 

Process is terminated.


### Misc.

In [ ]:
%%bash
mkdir /home/jupyter/.kaggle/
mv /home/jupyter/kaggle.json /home/jupyter/.kaggle/

In [ ]:
%%bash
kaggle datasets download --unzip -d phylake1337/fire-dataset

In [ ]:
%%bash
mv /home/jupyter/FYP_FIRENET/fire_dataset /home/jupyter/

In [ ]:
%%bash
gsutil -m mv -r /home/jupyter/fire_images/*.jpg gs://cloudfire_lyrical-edition-273206/fire_dataset/block3/

In [ ]:
%%bash
gsutil -m cp -r /home/jupyter/tfrecords-dataset-5xx gs://cloudfire_citric-sol_5670/fire_dataset/

In [ ]:
%%bash
gsutil -m cp -r /home/jupyter/fire_dataset/ gs://firetemple_avid-glass-5199/tb1

In [ ]:
%%bash
cd home/jupyter/FYP_FIRENET
python preprocess.py

### AI platform Train

- setup env variables

In [ ]:
import os
PROJECT = "citric-sol-273815" # REPLACE WITH YOUR PROJECT ID
BUCKET = "cloudfire_lyrical-edition-273206" # REPLACE WITH YOUR BUCKET NAME
MBUCKET = "firetemple_avid-glass-5199"
REGION = "us-central1" # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["MBUCKET"] = MBUCKET
os.environ["REGION"] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

- This allows ai platform to read/write to the staging bucket

In [ ]:
%%bash
# This command will fail if the Cloud Machine Learning Engine API is not enabled using the link above.
echo "Getting the service account email associated with the Cloud AI Platform API"

AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print (response['serviceAccount'])")  # If this command fails, the Cloud Machine Learning Engine API has not been enabled above.

echo "Authorizing the Cloud AI Platform account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$MBUCKET   
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$MBUCKET   # error message (if bucket is empty) can be ignored.  
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$MBUCKET   

- submits training job to the cloud

In [ ]:
%%bash
OUTDIR=gs://firetemple_avid-glass-5199/tb-event_files-aipf
REGION="us-west1"
TRAIN_DATA=gs://cloudfire_citric-sol_5670/fire_dataset/tfrecords-dataset-*x/
EVAL_DATA=gs://cloudfire_citric-sol_5670/fire_dataset/tfrecords-dataset-evalx/
JOBNAME=FireSage_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
# gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training FireSage_$(date -u +%y%m%d_%H%M%S) \
    --job-dir $OUTDIR \
    --runtime-version 2.1 \
    --python-version 3.7 \
    --module-name Trainer.config \
    --package-path ${PWD}/Trainer/ \
    --region $REGION \
    --scale-tier BASIC_GPU \
    -- \
    --output_dir $OUTDIR \
    --train_data_path $TRAIN_DATA \
    --eval_data_path $EVAL_DATA \
    --train_steps 500 \
    --learning_rate 0.001 \
    --batch_size 50
    

In [ ]:

%%bash
python --version

In [ ]:
%%bash
gcloud ai-platform jobs submit training --help

In [ ]:
from keras import backend as K 
def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
    activations = get_activations([X_batch,0]) 
    return activations

#### Managing Datasets

In [ ]:
pip install mxnet

In [ ]:
import requests

print('Beginning file download with requests')

url = 'https://www.floydhub.com/api/v1/resources/RwGobMM8FhzX9sUcgnBJTa?content=true&download=true&rename=ninaduf-datasets-no_fire-1'
r = requests.get(url)

with open('/home/jupyter/ninaduf-datasets-nofire-1.tar', 'wb') as f:
    f.write(r.content)

# Retrieve HTTP meta-data
print(r.status_code)
print(r.headers['content-type'])
print(r.encoding)

In [ ]:
import os
import tarfile

data_path = '/home/jupyter/Normal'
data_file = '/home/jupyter/ninaduf-datasets-nofire-1.tar'
if not os.path.exists(data_path):
    os.makedirs(data_path)
    with tarfile.open(data_file) as tar:
        tar.extractall(path=data_path)

In [5]:
%%bash
cd /home/jupyter/block5
find | wc -l
# ls  *.jpg

6710


In [22]:
%%bash
mv  /home/jupyter/block5/Normal/*.jpg /home/jupyter/Test/ 